In [8]:
# 확률적 경사하강법을 적용하여 영화 리뷰 데이터를 아래와 같은 절차로 머신러닝을 수행할 것임.
# 1) 확률적 경사하강법 버전의 로지스틱 회귀를 이용함
# 2) 영화 리뷰 데이터는 1000개씩 머신러닝을 수행함
# 3) 머신러닝을 위한 총 리뷰 데이터는 45000개로 함
# 4) 머신러닝 결과 테스트는 나머지 리뷰 데이터 5000개로 함
# 5) 머신러닝 결과는 파일로 저장함

import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
import pickle
import os
from mylib.progbar import ProgBar
from mylib.sgd_tokenizer import sgd_tokenizer

# path로 전달되는 정제된 영화 리뷰 데이터 파일을 읽고 실제 리뷰 텍스트와
# 그 라벨(긍정은 1, 부정은 0)을 리턴하는 제너레이터다.
# 함수라 하지 않고 제너레이터라고 하는 이유는 yield 키워드를 이용해 결과값을
# 반환해주기 때문이다.
# return : 결과값을 반환하고 함수를 종료함
# yield : 결과값을 반환하고 다음 호출 시 이어서 결과값을 생성함
# 또한 보통 제너레이터를 재호출할 때는 next()를 이용한다.
def stream_docs(path):
    with open(path, 'r') as f:
        # 파일 오픈한 후 최초로 next(f)한 것은 리뷰 데이터 첫번째 라인이
        # 제목(review, seniment)를 표시하는 부분이기 때문이다.
        next(f)
        # 리뷰 데이터 파일에서 한 줄씩 읽어 리뷰 텍스트와 라벨을 구분하고
        # 이를 반환함.
        for line in f:
            text, label = line[:-3], int(line[-2])
            yield text, label

# size로 제시되는 크기만큼 리뷰 데이터를 읽어 텍스트 데이터, 라벨을 리스트로
# 만들어 리턴함
def get_minibatch(doc_steam, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except Stoplteration:
        return None, None
    
    return docs, y

# 이전에 사용했던 CountVectorizer()나 TfidVectorizer()는 모든 데이터를 일괄적으로
# 메모리에 올려놓고 구동되는 함수다.
# 따라서 이 함수들은 지속적으로 유입되는 데이터에 대해서 부분적용할 수 없다.
# HashingVectorizer()는 해싱 기법을 이용하며, 데이터 독립적으로 구동되는
# 함수이므로, 이 함수를 이용하여 특성 벡터를 구성함.
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, tokenizer=sgd_tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
# doc_stream = stream_docs('./data/170925_movie_review_refiend.csv', encoding='CP949')
doc_stream = stream_docs('./data/170925_movie_review_refiend.csv')


pbar = ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    
    # X_train을 HashingVectorizer()를 이용해 특성 벡터로 변환함.
    X_train = vect.transform(X_train)
    
    # 확률적 경사하강법 버전의 로지스틱 회귀 SGDClassifier(loss='log', random_state=1, n_iter=1)의
    # partial_fit()을 이용해 부분 데이터 머신러닝을 수행함.
    # classes 인자는 가능한 y_train의 값을 numpy 배열로 지정해준다.
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' %clf.score(X_test, y_test))

curDir = os.getcwd()
dest = os.path.join(curDir, 'data', 'pklObject')

# 정확도는 이전 TfidVectorizer()을 이용해 선형 회귀한 머신러닝보다 떨어지지만
# 데이터의 용량과 관계없이 메모리에 대한 걱정을 하지 않고 머신러닝을 수행할 수 있다.
pickle.dump(clf, open(os.path.join(dest, 'SGDClassifier.pkl'), 'wb'), protocol=4)
print('머신러닝 데이터 저장 완료')

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


[######################]정확도: 0.867
머신러닝 데이터 저장 완료
